<a href="https://colab.research.google.com/github/neto-riga/Metodos_Numericos/blob/main/Ejercicio_11_Gauss_Seidel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Método de Gauss - Seidel
***Martínez Venegas Raúl***

***Rivera Gálvez Ernesto***

---



Resolver el siguiente sistema por el método de Gauss-Seidel:


$$\begin{eqnarray}
9x_{1} - x_{2} + 2x_{3} - 2x_{4} &=& -11 \\ -x_{1} + 8x_{2} - x_{3} + 3x_{4} &=& 15 \\ 2x_{1} - x_{2} + 10x_{3} - x_{4} &=& 45 \\ 2x_{1} + 3x_{2} - x_{3} + 8x_{4} &=& -39
\end{eqnarray}$$

* Tomar como vector inicial el vector de ceros.
* Estimar el error con la norma espectral.
* Comparación con el método de Jacobi.

El sistema en su forma matricial
$$
\begin{bmatrix}
9  & -1& 2& -2\\
-1 &  8& -1& 3\\
2  &  -1& 10& -1\\
2 & 3&  -1& 8
\end{bmatrix}
\cdot
\begin{bmatrix}
x_1\\
x_2\\
x_3\\
x_4
\end{bmatrix}
=
\begin{bmatrix}
 -11\\
 15\\
45\\
-39
\end{bmatrix}
$$

* [Método de Gauss - Seidel](#scrollTo=feAGSkqsd0dY)

  * [Empleo del método](#scrollTo=IoU9Ur6qJHyA)

  * [Estimación del Error con la Norma Espectral](#scrollTo=RUW5p2lgNP5r)

  * [Utilizando el Método de Jacobi](#scrollTo=Zbvi1bDqOcat)

    * [Conclusiones en la comparación de los métodos.](#scrollTo=mPyKTfY3REDw)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sympy as sp

In [ ]:
def printmat(m, x, b):
  print(" X\t\t", "\t"*int(n/2), 'A', '\t'*int(n/2 + n%2), 'B')
  for i in range(n):
    print(f'{x[i][0]:.4g}\t|', end='\t')
    for j in range(n):
      print(f'{m[i,j]:.4g}', end='\t')
    print(f'|{b[i][0]:.4g}')

## Empleo del método
---
Creamos los objetos de sus respectivos tamaños y los inicializamos. Iniciaremos el método con el vector inicial de ceros, es decir
$$
x_0^T = [0, 0, 0, 0]
$$

In [ ]:
n = 4

mat = np.eye(n)
x_prev = np.zeros(n).reshape(n,1)
x_sig = x_prev.copy()
x_sol = x_prev.copy()
b = np.zeros(n).reshape(n,1)

mat[0]= [9, -1, 2, -2]
b[0] = -11
mat[1] = [-1, 8, -1, 3]
b[1] = 15
mat[2]=[2, -1, 10, -1]
b[2] = 45
mat[3]=[2, 3, -1, 8]
b[3] = -39
R1 = 0 
R2 = 0

Ahora verificamos que la matriz sea EDD, de nos ser así, realizamos los cambios de columnas necesarios para intentar transformar la matriz en una EDD.

Si después de hacer los cambios correspondientes la matriz sigue sin ser EDD, entonces se obtiene un error.

In [ ]:
mayor = {
    'valor': 0,
    'columna': 0
  }
cam_x=[]
cam_p=[]
for i in range(n):
  mayor['valor'] = 0
  for k in range(n):
    if abs(mat[i,k]) > abs(mayor.get('valor')): # Encuentra el mayor elemento
      mayor['valor'] = mat[i,k] # Guarda el valor del elemento
      mayor['columna'] = k # Guarda la columna en la que se encuentra
  if abs(mayor['valor']) != abs(mat[i,i]): # Si el i,i no es el mayor
    mat[:,[k, mayor.get('columna')]] = mat[:,[mayor.get('columna'), k]] # intercambiar las columnas
    cam_x.append(mayor.get('columna'))
    cam_p.append(k)
for i in range(n):
  suma_resto = 0
  for j in range(n):
    if j != i:
      suma_resto += abs(mat[i, j])
  if abs(mat[i, i]) < suma_resto:
    raise ValueError("Su matriz no pudo hacerse EDD")
print("Matriz EDD")
printmat(mat,x_prev,b)

x_frame = x_prev.copy()
for iter in range(7):
  for i in range(n):
    R1=0
    R2=0
    for j in range(n):
      if j < i:
        R1 += (mat[i,j]/mat[i,i]) * x_sig[j]
        
      elif j > i:
        R2 += (mat[i,j]/mat[i,i]) * x_prev[j]
        
    x_sig[i] = -R1 - R2 + (b[i][0]/mat[i,i])
  x_sol = x_sig.copy()
  for g in range(len(cam_x)):
    x_sol[[cam_x[g-1], cam_p[g-1]]]=x_sol[[cam_p[g-1], cam_x[g-1]]]
  x_frame = np.append(x_frame, x_sol.copy(), axis=1)
  x_prev = x_sig.copy()

Matriz EDD
 X		 		 A 		 B
0	|	9	-1	2	-2	|-11
0	|	-1	8	-1	3	|15
0	|	2	-1	10	-1	|45
0	|	2	3	-1	8	|-39


Después de obtener una matriz EDD se muestra el comportamiento de la iteraciones

In [ ]:
nombres = [f'x{i}' for i in range(len(x_frame[0]))]
X = pd.DataFrame(x_frame,
                 columns=nombres)
X

,x0,x1,x2,x3,x4,x5,x6,x7
0,0.0,-1.222222,-3.145833,-3.006820,-3.002317,-3.000319,-3.000055,-3.000009
1,0.0,1.722222,3.821615,3.964395,3.994750,3.999126,3.999860,3.999977
2,0.0,4.916667,5.051259,5.008780,5.001554,5.000251,5.000041,5.000007
3,0.0,-4.600694,-4.890240,-4.983846,-4.997258,-4.999561,-4.999928,-4.999988


Se muestra la aproximación del vector solución de la pultima iteración

In [ ]:
R = sp.Matrix(x_sol)
R.evalf(3)

Matrix([
[-3.0],
[ 4.0],
[ 5.0],
[-5.0]])

## Estimación del Error con la Norma Espectral
---
Mostrando el error por iteración en las últimas 5 iteraciones.

In [ ]:
error = []
for i in range(len(x_frame[0])-5,len(x_frame[0])):
  error.append(x_frame[:, (i)] - x_frame[:,(i-1)])
for i in range(5):
  mayor = 0
  for j in error[i]:
    if abs(mayor)<abs(j):
      mayor = j
  error[i]= np.append(error[i], abs(mayor))
nombres2 = [f'x{i} - x{i-1}' for i in range(len(x_frame[0])-5,len(x_frame[0]))]
X1 = pd.DataFrame(error,
                 columns=nombres2)
inx = [f'{i+1}' for i in range(n)]
inx.append('ɛ')
X1['Valores'] = inx
X1.set_index('Valores', inplace=True)
X1

,x3 - x2,x4 - x3,x5 - x4,x6 - x5,x7 - x6
Valores,,,,,
1,0.139013,0.142780,-0.042479,-0.093606,0.142780
2,0.004503,0.030355,-0.007226,-0.013412,0.030355
3,0.001998,0.004376,-0.001303,-0.002303,0.004376
4,0.000264,0.000734,-0.000210,-0.000367,0.000734
ɛ,0.000047,0.000117,-0.000034,-0.000060,0.000117


## Utilizando el Método de Jacobi
---


In [ ]:
mat = np.eye(n)
x_prev = np.zeros(n).reshape(n,1)
x_sig = x_prev.copy()
x_sol = x_prev.copy()
b = np.zeros(n).reshape(n,1)

mat[0]= [9, -1, 2, -2]
b[0] = -11
mat[1] = [-1, 8, -1, 3]
b[1] = 15
mat[2]=[2, -1, 10, -1]
b[2] = 45
mat[3]=[2, 3, -1, 8]
b[3] = -39

In [ ]:
mayor = {
    'valor': 0,
    'columna': 0
  }
cam_x=[]
cam_p=[]
for i in range(n):
  mayor['valor'] = 0
  for k in range(n):
    if abs(mat[i,k]) > abs(mayor.get('valor')): # Encuentra el mayor elemento
      mayor['valor'] = mat[i,k] # Guarda el valor del elemento
      mayor['columna'] = k # Guarda la columna en la que se encuentra
  if abs(mayor['valor']) != abs(mat[i,i]): # Si el i,i no es el mayor
    mat[:,[k, mayor.get('columna')]] = mat[:,[mayor.get('columna'), k]] # intercambiar las columnas
    cam_x.append(mayor.get('columna'))
    cam_p.append(k)
for i in range(n):
  suma_resto = 0
  for j in range(n):
    if j != i:
      suma_resto += abs(mat[i, j])
  if abs(mat[i, i]) < suma_resto:
    raise ValueError("Su matriz no es EDD")

x_frame = x_prev.copy()
for iter in range(7):

  for i in range(n):
    x_sig[i] = b[i][0] / mat[i, i]
    for j in range(n):
      if j != i:
        x_sig[i] -= mat[i,j] / mat[i, i] * x_prev[j][0]
  x_sol = x_sig.copy()
  for g in range(len(cam_x)):
    x_sol[[cam_x[g-1], cam_p[g-1]]]=x_sol[[cam_p[g-1], cam_x[g-1]]]
  x_frame = np.append(x_frame, x_sol.copy(), axis=1)
  x_prev = x_sig.copy()

In [ ]:
nombres = [f'x{i}' for i in range(len(x_frame[0]))]
X = pd.DataFrame(x_frame,
                 columns=nombres)
X

,x0,x1,x2,x3,x4,x5,x6,x7
0,0.0,-1.222222,-3.097222,-2.799576,-3.053853,-2.971278,-3.011707,-2.995090
1,0.0,1.875000,4.112847,3.809679,4.065315,3.974012,4.013429,3.995342
2,0.0,4.500000,4.444444,5.059722,4.932137,5.020175,4.989706,5.004193
3,0.0,-4.875000,-4.710069,-5.087457,-4.971270,-5.019512,-4.994913,-5.003396


In [ ]:
R = sp.Matrix(x_sol)
R.evalf(3)

Matrix([
[-3.0],
[ 4.0],
[ 5.0],
[-5.0]])

In [ ]:
error = []
for i in range(len(x_frame[0])-5,len(x_frame[0])):
  error.append(x_frame[:, (i)] - x_frame[:,(i-1)])
for i in range(5):
  mayor = 0
  for j in error[i]:
    if abs(mayor)<abs(j):
      mayor = j
  error[i]= np.append(error[i], abs(mayor))
nombres2 = [f'x{i} - x{i-1}' for i in range(len(x_frame[0])-5,len(x_frame[0]))]
X1 = pd.DataFrame(error,
                 columns=nombres2)
inx = [f'{i+1}' for i in range(n)]
inx.append('ɛ')
X1['Valores'] = inx
X1.set_index('Valores', inplace=True)
X1

,x3 - x2,x4 - x3,x5 - x4,x6 - x5,x7 - x6
Valores,,,,,
1,0.297647,-0.303168,0.615278,-0.377387,0.615278
2,-0.254278,0.255636,-0.127585,0.116186,0.255636
3,0.082575,-0.091303,0.088038,-0.048242,0.091303
4,-0.040429,0.039417,-0.030470,0.024599,0.040429
ɛ,0.016617,-0.018087,0.014488,-0.008483,0.018087


### Conclusiones en la comparación de los métodos.
---
Notemos que ambos métodos llegan al mismo resultado, sin embargo, podemos ver claramente la diferencia en la que convergen en sus errores y la rapidez con la que el método converge. Haremos una comparación en la siguiente tabla comparando la iteración en la que llegan al resultado redondeando a dos decimales:

In [ ]:
comparacion = pd.DataFrame({'Iteración': [3, 5],
              'Error Espectral': [4.7e-5, 1.448e-3],
              })
comparacion['Método'] = ['Gauss-Seidel', 'Jacobi']
comparacion.set_index('Método', inplace=True)
comparacion

,Iteración,Error Espectral
Método,,
Gauss-Seidel,3,0.000047
Jacobi,5,0.001448


Donde queda clara la ventaja de usar Gauss-Seidel para esta matriz en específico, pues llego a la solución en dos iteraciones menos con un error mucho menor.